In [1]:
"""
Created on Tue Dec 19 07:16:52 2017

@author: Verena Yacoub

Attributing codes and external references
[1] https://github.com/davidawad/Lane-Detection/blob/master/detection.py (used in calculating the best fit line)
[2] udacity codes provided throughout classes (used for the fundamental methods in image processing)
[3] https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.itemset.html (helped in elimination of horizontal lines)
[4] https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html (for video reading and frames extraction)
"""

import numpy as np
import cv2

############### Reference [4] (see the block comment in the top) #################
cap = cv2.VideoCapture('challenge.mp4')
cv2.namedWindow('frames')
cv2.namedWindow('raw lines')
while(cap.isOpened()):
    ret, image = cap.read()

    if ret is True:
############### Reference [2] (see the block comment in the top) #################        
        gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY) #convert to gray scale
        
             
        kernel_size = 5 #define kernel size for gaussian smoothing
        blur = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0) #performing smoothing or noise reduction
             
        
        lowthresh = 70 # minimum value below which canny filter classify a signal as non edge
        highthresh = 190# maximum value above which a signal is a sure edge
        edges = cv2.Canny(blur, lowthresh, highthresh) #apply canny edge detection
             
             
        mask = np.zeros_like(edges)   #create a blank/dark image with the same dimensions as original
        inmaskcolour = 255 # defining the color of the mask drawn on the dark image (here white)  
        
        
        imshape = image.shape # get the dimensions of the original image
        vertices = np.array([[(0,imshape[0]),((imshape[1]/3)+100, imshape[0]-(imshape[0]/2.6)), ((imshape[1]/3)+300,imshape[0]-(imshape[0]/2.6)), (imshape[1],imshape[0])]], dtype=np.int32) #defining the region of interest
        cv2.fillPoly(mask, vertices, inmaskcolour) # generate the polygon with the above vertices and fill it in white    
        
            
        masked_edges = cv2.bitwise_and(edges, mask) #returning the image only where mask pixels are nonzero
              
        
        # Define the Hough transform parameters
        rho=2           # distance resolution in pixels of the Hough grid
        theta = np.pi/180   # angular resolution in radians of the Hough grid
        threshold= 15       # minimum number of votes (intersections in Hough grid cell)
        min_line_length= 20       # minimum number of pixels making up a line
        max_line_gap= 20       # maximum gap in pixels between connectable line segments
        
        img= np.copy(image)*0  # again create a blank image with the same original dimensions to draw lines on it    
        lines= cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), minLineLength=min_line_length, maxLineGap=max_line_gap) # applying probabilistic hough transform
        
        line_image= np.copy(image)*0 # create a new blank image for rawlines to be drawn
        ii=0
        for line in lines: #iterate over the generated houghlines and drawing them
             for x1,y1,x2,y2 in line:
                 if abs(y2-y1)>10:#this statement to eliminate horizontal lines as they can effect the slope of the best fit
                    cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10) #draw raw hough lines on the blank image
                    i=ii #this counter to know the size of the array of vertical lines only
                    ii +=1 
        lines_raw = cv2.addWeighted(image, 0.8, line_image, 1, 0) # combine the image with the drawn raw lines with the original image
        cv2.imshow('raw lines', lines_raw) # show the combined image
 
 ############### Reference [3] (see the block comment in the top) #################      
        v_array = np.empty((i+1,1,4)) #creating empty array with the same size of the number of vertical lines only
        j=0 # create the counter to fill the array
        for line in lines: 
             
            for x1,y1,x2,y2 in line:
                  if abs(y2-y1)>10:# iterate again with the same condition to fill the empty array with the desired values
                    v_array.itemset((j,0,0),x1) # fill the vertical array itemwise
                    v_array.itemset((j,0,1),y1) # fill the vertical array itemwise
                    v_array.itemset((j,0,2),x2) # fill the vertical array itemwise
                    v_array.itemset((j,0,3),y2) # fill the vertical array itemwise
                    j +=1
        v_array=v_array.real.astype(int) # make the array of integer values as the default is float
                  
############### Reference [1] (see the block comment in the top) #################                  
        thickness=15 # set the thickness of lines to be drawn
        colour=[0,0,255] # set the colour of lines o be drawn      
      
        v_array = v_array.reshape(v_array.shape[0], v_array.shape[2])   # reshape lines to a 2d matrix
        
            
        v_array = v_array[~np.isnan(v_array) & ~np.isinf(v_array)] # remove junk from array
        v_array.shape = (v_array.shape[0]//2,2) # split the x1,y1 and x2,y2 in separate cells
       
            
        right_lines = np.array(list(filter(lambda x: x[0] > (img.shape[1]/2), v_array))) #filter points in the right half of the image in a list
        
        max_right_x, max_right_y = right_lines.max(axis=0) # find max value in the right half
        min_right_x, min_right_y = right_lines.min(axis=0) # find min value in the right half
        
            
        left_lines = np.array(list(filter(lambda x: x[0] < (img.shape[1]/2), v_array))) #filter points in the right half of the image in a list
        max_left_x, max_left_y = left_lines.max(axis=0) # find max value in the left half
        min_left_x, min_left_y = left_lines.min(axis=0) # find min value in the right half
        
            
        rightbestfit = np.poly1d(np.polyfit(right_lines[:,1], right_lines[:,0], 2)) # best fit of the right points
        leftbestfit  = np.poly1d(np.polyfit(left_lines[:,1], left_lines[:,0], 2)) # best fit of the left points
        
            
        min_absolute_y = min(min_left_y, min_right_y) # min vertical value of both image halves so the lines end at the same horizon
        
           
        max_right_x = int(rightbestfit(img.shape[0])) # find the x value on the fit line at the bottom of image
        min_right_x = int(rightbestfit(min_right_y)) # find the x value on the fit line at the horizon previously set
        
        min_left_x = int(leftbestfit(img.shape[0]))  # find the x value on the fit line at the bottom of image
        
        r1 = (min_right_x, min_absolute_y) # define upper point of the right fitting line
        r2 = (max_right_x, img.shape[0]) # define bottom point of the right fitting line
        print('Right points r1 and r2,', r1, r2)
        cv2.line(img, r1, r2, colour, thickness) # draw right line on the blank image
        
        l1 = (max_left_x, min_absolute_y) # define upper point of the left fitting line
        l2 = (min_left_x, img.shape[0]) # define bottom point of the left fitting line
        print('Left points l1 and l2,', l1, l2)
        cv2.line(img, l1, l2, colour, thickness) # draw left line on the blank image
          
############### Reference [2 & 4] (see the block comment in the top) #################        
        lines_edges = cv2.addWeighted(image, 0.8, img, 1, 0) # combine the image with the drawn lines with the original image
        cv2.imshow('frames', lines_edges) # show the combined image
        k= cv2.waitKey(5) & 0xFF
    
        if k==15:
         break

cv2.destroyAllWindows()
cap.release()  




